In [2]:
import numpy as np
import pandas as pd
import os
from haversine import haversine

In [3]:
def project(lat,long):
    lat = np.radians(lat)
    long = np.radians(long)
    x = (128/np.pi)*(2**17)*(long + np.pi)
    y = (128/np.pi)*(2**17)*(np.pi - np.log(np.tan(np.pi/4 + lat/2)))
    return x,y

In [4]:
def inverse_project(x, y):
    F = 128 / np.pi * 2 ** 17
    lng = (x / F) - np.pi
    lat = (2 * np.arctan(np.exp(np.pi - y / F)) - np.pi / 2)
    lng = lng * 180 / np.pi
    lat = lat * 180 / np.pi
    return lat, lng

In [13]:
def process_bounding_box(file_path):
    filename = os.path.basename(file_path)
    lat_str, lon_str = os.path.splitext(filename)[0].split(',')
    lat = float(lat_str)
    lon = float(lon_str)
    
    print(lat, lon)
    
    lat_lon_list = []  # Initialize a list to store lat-lon pairs
    
    with open(file_path, 'r') as file:
        lines = file.readlines()  # Read all lines in the file
        for line in lines:
            values = line.strip().split(' ')
            prediction = float(values[0])
            corners = [float(value) for value in values[1:5]]

            x_c, y_c, w, h = corners
            print(x_c, y_c, w, h)

            result = project(lat, lon)
            print(result)
            delta_y = y_c*1120 - 560
            delta_x = x_c*1120 - 560
            print(delta_x, delta_y)
            lat_value, lng_value = inverse_project(result[0] + delta_x, result[1] + delta_y)

            # Append the lat-lon pair to the list
            lat_lon_list.append((lat_value, lng_value))
    
    return lat_lon_list


In [14]:
process_bounding_box("/home/patel_zeel/compass24/corrected_txt/punjab/30.70,75.33.txt")

30.7 75.33
0.908398 0.757168 0.0376248 0.0837074
(23798480.896, 13768119.915674092)
457.4057600000001 288.02815999999996


[(30.69734284198712, 75.33490743141175)]

In [31]:
def process_directory(directory_path):
    lat_lon_list = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        lat_lon = process_bounding_box(file_path)
        lat_lon_list.extend(lat_lon)
    
    return lat_lon_list

In [67]:
directory = ("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/YOLO_LOCALIZATION/runs/detect/punjab_pp/labels_copy")

In [68]:
output_dir = process_directory(directory)

In [69]:
output_dir = sorted(output_dir, key=lambda x: (x[0], x[1]))

In [70]:
output_dir

[(29.60109779771108, 75.24556879844668),
 (29.601102457516784, 75.24554175968174),
 (29.616212593016915, 75.2158486219406),
 (29.62134098009779, 75.22895261154177),
 (29.621785963894904, 75.21537805175785),
 (29.623272773600338, 75.21414382551198),
 (29.62333408982341, 75.23029244060518),
 (29.625522588753206, 75.25819833660125),
 (29.62624316649819, 75.22712428398133),
 (29.6273454937174, 75.25904456024168),
 (29.62878373512373, 75.30336040534974),
 (29.631790403251124, 75.29651288280488),
 (29.631991789507712, 75.29730478076937),
 (29.63275113354691, 75.30064404945377),
 (29.63552284526118, 75.29326867294307),
 (29.639149784749183, 75.31936195869444),
 (29.640156847307804, 75.2967736003876),
 (29.641527000189836, 75.29626426563264),
 (29.72640627510006, 75.31990233154292),
 (29.730754211138134, 75.31736595573422),
 (29.733374090805206, 75.31885577220913),
 (29.751777375107963, 75.57675069732664),
 (29.756480717422075, 75.2342505158043),
 (29.757074789211657, 75.25876413593288),
 (29.

In [71]:
def remove_close_coordinates(coordinates, min_distance=53):
    # List to store filtered coordinates
    filtered_coordinates = []
    
    # Iterate over the sorted list of coordinates
    for i in range(len(coordinates)):
        # If it's the first coordinate, add it to the filtered list
        if i == 0:
            filtered_coordinates.append(coordinates[i])
        else:
            # Calculate the distance between the current and previous coordinates using Haversine formula
            distance = haversine(coordinates[i], coordinates[i-1], unit='m')
            
            # If the distance is greater than or equal to the minimum distance, add the coordinate to the filtered list
            if distance >= min_distance:
                filtered_coordinates.append(coordinates[i])
            else:
                # Print the pairs of kilns from which one is being removed
                print(f"kiln_out: {coordinates[i]} kiln_at: {coordinates[i-1]} distance:{distance}")
    
    return filtered_coordinates


In [72]:
filtered_coordinates = remove_close_coordinates(output_dir)

kiln_out: (29.601102457516784, 75.24554175968174) kiln_at: (29.60109779771108, 75.24556879844668) distance:2.665031016309517
kiln_out: (29.774811035909934, 75.94498473224641) kiln_at: (29.774809253431243, 75.94497884395601) distance:0.601882326664356
kiln_out: (29.775016599223008, 75.94487575494767) kiln_at: (29.774811035909934, 75.94498473224641) distance:25.16146560098079
kiln_out: (29.775017558776558, 75.94497222328188) kiln_at: (29.775016599223008, 75.94487575494767) distance:9.3112850072925
kiln_out: (29.795130799519573, 75.99614532833101) kiln_at: (29.79507028430495, 75.99616475868224) distance:6.9853276399630975
kiln_out: (29.79513924606506, 75.99584119377138) kiln_at: (29.795130799519573, 75.99614532833101) distance:29.362775507786466
kiln_out: (29.796757643159, 75.53493449211119) kiln_at: (29.796754702557504, 75.53494437160491) distance:1.0078327352629706
kiln_out: (29.805325120139035, 75.42844491100313) kiln_at: (29.80506965142431, 75.42844750652311) distance:28.4079681000834

In [73]:
len(output_dir)

2809

In [74]:
len(filtered_coordinates)

2275

In [ ]:
process_bounding_box("/home/rishabh.mondal/Brick-Kilns-project/albk_rishabh/albk_v2/yolo_tp/varified_true_positives/24.78,84.03.txt")

Prediction: 0.0, 24.78412999801193,84.02686524868015


In [75]:
df = pd.DataFrame(filtered_coordinates)

In [76]:
df.to_csv('punjab_pp.csv', index=False)

In [41]:
def rename_csv_file(old_file_path, new_file_name):
    # Get the directory path and the file extension from the old file path
    directory, old_file_name = os.path.split(old_file_path)
    file_name, file_extension = os.path.splitext(old_file_name)
    
    # Construct the new file path with the new file name and the same directory
    new_file_path = os.path.join(directory, new_file_name + file_extension)
    
    # Rename the file
    os.rename(old_file_path, new_file_path)
    
    print(f"File renamed to: {new_file_path}")

In [42]:
rename_csv_file("/home/shahshrimay/compliance/output_vtp.csv", "output_varified_true_positives")

File renamed to: /home/shahshrimay/compliance/output_varified_true_positives.csv
